In [1]:
import os
import numpy as np
import nltk
from keras.models import Sequential, Model
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, recurrent, Embedding,Dropout, Bidirectional, Input, merge
from keras.preprocessing.sequence import pad_sequences
import keras
import json
from data_utils import *
from model import *

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
SENTENCE_LENGTH = 20
VOCAB_SIZE = 10000

def load_cornell_movie_data(start_line, limit, vocab_dict):
    sentences = read_lines("movie_lines.txt", " +++$+++ ", 3, 4, start_line, limit)
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

def load_twitter_data(start_line, limit, vocab_dict):
    sentences = read_sentences("ShortenTwitterAsciiCorpus.txt")[1000:2000]
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

def load_friends_data():
    sentences = read_lines("friends-final.txt", "	", 2, 5, start_line = 0, limit = 10000)
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

In [3]:
#model = create_model(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 100, 3)
model = create_model_with_attention(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 300, 100, 3)
print(model.summary())
model.load_weights("0new_model.h5")

TypeError: Layer permute_1 does not support masking, but was passed an input_mask: Tensor("embedding_1/NotEqual:0", shape=(?, 20), dtype=bool)

In [ ]:
vocab_dict, idx_dict = load_vocab("vocab_dict.json")

In [ ]:
batch_size = 1000
start_point = 8000
batch = 8
round = 0
while True:
    print("Getting data:", start_point, batch_size)
    x_vec, y_vec = load_cornell_movie_data(start_point, batch_size, vocab_dict)
    y = np.argmax(y_vec, axis=2)
    print(" ".join([idx_dict[x] for x in y[0] ]) )
    
    if len(x_vec) == 0:
        start_point = 0
        batch = 0
        x_vec, y_vec = load_cornell_movie_data(start_point, batch_size, vocab_dict)
        round += 1 
    model.fit(x_vec, y_vec, batch_size=100, nb_epoch=100, shuffle = True)
    model.save_weights(str(batch%2)+"new_model.h5")
    start_point += batch_size
    batch += 1
    print("--------------------------------------------------------------------------")
    print("Round:", round, "Batch:", batch)
    print("--------------------------------------------------------------------------")

In [ ]:
#do you clean
sen = "hi"
sen = sen.lower().replace('?', '').replace('!', '').replace( '.', '').split(" ")
vec = sentence_to_vec([sen], vocab_dict, 'UNK', SENTENCE_LENGTH)
#print(model.predict(vec).shape)
res = model.predict(vec)
print(res)
vec_y = np.argmax(res, axis=2)
" ".join([idx_dict[x] for x in vec_y[0] ])
#if idx_dict[x]!= 'UNK'